In [3]:
import time
import requests
import json
from pprint import pprint
import pandas as pd
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime

In [5]:
weather_url = 'https://api.met.no/weatherapi/locationforecast/2.0/complete?lat=69.64&lon=18.95'
weather_data = requests.get(weather_url).json()


### Look at the data structure

In [6]:
pprint(weather_data)

{'geometry': {'coordinates': [18.95, 69.64, 0], 'type': 'Point'},
 'properties': {'meta': {'units': {'air_pressure_at_sea_level': 'hPa',
                                   'air_temperature': 'celsius',
                                   'air_temperature_max': 'celsius',
                                   'air_temperature_min': 'celsius',
                                   'air_temperature_percentile_10': 'celsius',
                                   'air_temperature_percentile_90': 'celsius',
                                   'cloud_area_fraction': '%',
                                   'cloud_area_fraction_high': '%',
                                   'cloud_area_fraction_low': '%',
                                   'cloud_area_fraction_medium': '%',
                                   'dew_point_temperature': 'celsius',
                                   'fog_area_fraction': '%',
                                   'precipitation_amount': 'mm',
                                   'p

In [7]:
weather_data['geometry']


{'type': 'Point', 'coordinates': [18.95, 69.64, 0]}

In [8]:
weather_data['properties']['meta']

{'updated_at': '2022-08-19T09:36:51Z',
 'units': {'air_pressure_at_sea_level': 'hPa',
  'air_temperature': 'celsius',
  'air_temperature_max': 'celsius',
  'air_temperature_min': 'celsius',
  'air_temperature_percentile_10': 'celsius',
  'air_temperature_percentile_90': 'celsius',
  'cloud_area_fraction': '%',
  'cloud_area_fraction_high': '%',
  'cloud_area_fraction_low': '%',
  'cloud_area_fraction_medium': '%',
  'dew_point_temperature': 'celsius',
  'fog_area_fraction': '%',
  'precipitation_amount': 'mm',
  'precipitation_amount_max': 'mm',
  'precipitation_amount_min': 'mm',
  'probability_of_precipitation': '%',
  'probability_of_thunder': '%',
  'relative_humidity': '%',
  'ultraviolet_index_clear_sky': '1',
  'wind_from_direction': 'degrees',
  'wind_speed': 'm/s',
  'wind_speed_of_gust': 'm/s',
  'wind_speed_percentile_10': 'm/s',
  'wind_speed_percentile_90': 'm/s'}}

In [9]:
weather_data['properties']['timeseries']

[{'time': '2022-08-19T09:00:00Z',
  'data': {'instant': {'details': {'air_pressure_at_sea_level': 1013.5,
     'air_temperature': 16.1,
     'air_temperature_percentile_10': 15.5,
     'air_temperature_percentile_90': 16.7,
     'cloud_area_fraction': 99.0,
     'cloud_area_fraction_high': 98.8,
     'cloud_area_fraction_low': 1.8,
     'cloud_area_fraction_medium': 15.0,
     'dew_point_temperature': 9.4,
     'fog_area_fraction': 0.0,
     'relative_humidity': 66.5,
     'ultraviolet_index_clear_sky': 2.2,
     'wind_from_direction': 346.9,
     'wind_speed': 1.0,
     'wind_speed_of_gust': 2.4,
     'wind_speed_percentile_10': 0.6,
     'wind_speed_percentile_90': 1.6}},
   'next_12_hours': {'summary': {'symbol_code': 'cloudy',
     'symbol_confidence': 'somewhat certain'},
    'details': {'probability_of_precipitation': 19.8}},
   'next_1_hours': {'summary': {'symbol_code': 'cloudy'},
    'details': {'precipitation_amount': 0.0,
     'precipitation_amount_max': 0.0,
     'precipita

In [20]:
weather_data['properties']['timeseries'][0]['data']['next_6_hours']['details']

{'air_temperature_max': 17.8,
 'air_temperature_min': 17.0,
 'precipitation_amount': 0.0,
 'precipitation_amount_max': 0.0,
 'precipitation_amount_min': 0.0,
 'probability_of_precipitation': 0.3}

### Read out the desired lines and put them into a dataframe for easier handling

In [25]:

def precip_prob(i):
    try:
        return weather_data['properties']['timeseries'][i]['data']['next_1_hours']['details']['probability_of_precipitation']
    except:
        None
def precip_amount(i):
    try:
        return weather_data['properties']['timeseries'][i]['data']['next_1_hours']['details']['precipitation_amount']
    except:
        None

In [26]:
data = [weather_data['properties']['timeseries'][i]['data']['instant']['details'] for i in range(len(weather_data['properties']['timeseries']))]
times = [weather_data['properties']['timeseries'][i]['time'] for i in range(len(weather_data['properties']['timeseries']))]
precip_proba = [precip_prob(i) for i in range(len(weather_data['properties']['timeseries']))]
precip = [precip_amount(i) for i in range(len(weather_data['properties']['timeseries']))]
df = pd.DataFrame(data, index=times)
df.index = pd.to_datetime(df.index)
df.index.name = 'forecast_time'
df['lat'] = weather_data['geometry']['coordinates'][1]
df['lon'] = weather_data['geometry']['coordinates'][0]
df['height'] = weather_data['geometry']['coordinates'][2]
df['timestamp'] = weather_data['properties']['meta']['updated_at']
df['precipitation_prob'] = precip_proba
df['precipitation_amount'] = precip


In [27]:
df

,air_pressure_at_sea_level,air_temperature,air_temperature_percentile_10,air_temperature_percentile_90,cloud_area_fraction,cloud_area_fraction_high,cloud_area_fraction_low,cloud_area_fraction_medium,dew_point_temperature,fog_area_fraction,...,wind_speed,wind_speed_of_gust,wind_speed_percentile_10,wind_speed_percentile_90,lat,lon,height,timestamp,precipitation_prob,precipitation_amount
forecast_time,,,,,,,,,,,,,,,,,,,,,
2022-08-19 09:00:00+00:00,1013.5,16.1,15.5,16.7,99.0,98.8,1.8,15.0,9.4,0.0,...,1.0,2.4,0.6,1.6,69.64,18.95,0,2022-08-19T09:36:51Z,0.0,0.0
2022-08-19 10:00:00+00:00,1013.2,17.0,16.2,17.8,97.1,96.5,6.0,14.3,10.3,0.0,...,1.0,2.2,0.8,1.6,69.64,18.95,0,2022-08-19T09:36:51Z,0.1,0.0
2022-08-19 11:00:00+00:00,1012.9,17.8,16.8,18.9,99.6,99.0,8.7,67.2,11.2,0.0,...,1.0,2.3,0.7,3.0,69.64,18.95,0,2022-08-19T09:36:51Z,0.0,0.0
2022-08-19 12:00:00+00:00,1012.6,17.4,16.6,18.9,100.0,99.8,15.1,99.6,11.2,0.0,...,1.0,2.2,0.6,2.4,69.64,18.95,0,2022-08-19T09:36:51Z,0.1,0.0
2022-08-19 13:00:00+00:00,1012.4,17.2,16.5,19.3,99.9,95.0,21.3,98.2,11.9,0.0,...,0.9,1.8,0.5,2.8,69.64,18.95,0,2022-08-19T09:36:51Z,1.7,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-27 06:00:00+00:00,1012.3,10.0,8.4,12.1,100.0,37.5,100.0,100.0,9.1,NaN,...,4.9,NaN,4.2,5.6,69.64,18.95,0,2022-08-19T09:36:51Z,NaN,NaN
2022-08-27 12:00:00+00:00,1013.9,10.9,9.4,15.6,100.0,10.9,95.3,99.2,9.2,NaN,...,5.0,NaN,4.5,5.3,69.64,18.95,0,2022-08-19T09:36:51Z,NaN,NaN
2022-08-27 18:00:00+00:00,1017.4,10.2,8.0,14.1,100.0,19.5,97.3,92.6,8.7,NaN,...,5.7,NaN,5.2,5.9,69.64,18.95,0,2022-08-19T09:36:51Z,NaN,NaN


## Test the database push

In [28]:
org = "FHSWF"
bucket = "weather"
token="clmwcDxm_39ML529Q5lbtT4XRLChNqqGull5VJt_zrFkAw4rTTPcMyzkhPo2sAlDhr6mIJW66en1PqRGmybRXQ=="
with InfluxDBClient("http://localhost:8086", token=token, org=org) as dbclient:
    write_api = dbclient.write_api(write_options=SYNCHRONOUS)
    write_api.write(bucket, record=df, data_frame_measurement_name=df['timestamp'][0], data_frame_tag_columns='forecast')